In [1]:
%matplotlib inline
from __future__ import print_function
import os
from pyspark import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as sql
import pyspark.sql.types as types
#from pyspark.sql.functions import udf, length
import matplotlib.pyplot as plt
import numpy
import math
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import pyspark.ml.feature as feature

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
# Load Processed Parquet
sqlContext = SQLContext(sc)
ols = sqlContext.read.json("../data/idigbio_envo_terms.json")
total_records = ols.count()
print(total_records)
ols.printSchema()


74130
root
 |-- noun: string (nullable = true)
 |-- response: struct (nullable = true)
 |    |-- description: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- iri: string (nullable = true)
 |    |-- is_defining_ontology: boolean (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- obo_id: string (nullable = true)
 |    |-- ontology_name: string (nullable = true)
 |    |-- ontology_prefix: string (nullable = true)
 |    |-- short_form: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- term_id: string (nullable = true)
 |-- term_label: string (nullable = true)
 |-- time: string (nullable = true)



In [10]:
print(ols.filter(sql.col("term_id") != "").count())
print(ols.filter(sql.col("term_id") == "").count())

19131
54999


In [12]:
# Manually review a random sample of positive results and negative results
ols\
    .filter(sql.col("term_id") != "")\
    .sample(withReplacement=False, fraction=0.01)\
    .unionAll(
    ols\
        .filter(sql.col("term_id") == "")\
        .sample(withReplacement=False, fraction=0.002)
    )\
    .select(sql.col("noun"), sql.col("term_label"))\
    .show(1000, truncate=False)

+----------------------+------------------------------------------------+
|noun                  |term_label                                      |
+----------------------+------------------------------------------------+
|fin                   |paired limb/fin segment                         |
|ponds.                |intermittent pond                               |
|t-42mm                |process                                         |
|3'x4                  |arsenate treated wood                           |
|g-13306               |saline hydrographic feature                     |
|dupl.=5               |karst field                                     |
|brown-headed          |brown sugar                                     |
|11/16                 |industrial wastewate treatment plant            |
|web                   |cardiac mesoderm                                |
|1.x.1982              |partially_surrounded_by                         |
|51.                   |siltstone     

In [16]:
# Need to filter out terms that were the result of mapping a numerical string
# criteria is perhaps more than one number in a string? (remember nouns had to
# be >2 chars to try to map)

In [15]:
# What kinds of things were we finding?
terms = ols\
    .groupBy(sql.col("term_id"), sql.col("term_label"))\
    .count()\
    .orderBy(sql.col("count"), ascending=False)
    
print(terms.count())
terms.show(10, truncate=False)
    


2503
+--------------------------------------------------+------------------------------------------+-----+
|term_id                                           |term_label                                |count|
+--------------------------------------------------+------------------------------------------+-----+
|                                                  |                                          |54999|
|envo:http://purl.obolibrary.org/obo/ENVO_00000017 |saline hydrographic feature               |442  |
|envo:http://purl.obolibrary.org/obo/PATO_0002224  |V-shaped                                  |398  |
|envo:http://purl.obolibrary.org/obo/UBERON_0012175|acoustico-facial VII-VIII ganglion complex|282  |
|envo:http://purl.obolibrary.org/obo/ENVO_00002258 |loam                                      |240  |
|envo:http://purl.obolibrary.org/obo/ENVO_01000483 |flat glass                                |207  |
|envo:http://purl.obolibrary.org/obo/ENVO_01000121 |colloidal sediment       